## 콘텐츠 기반
- 사용자가 주로 이용하는 콘텐츠의 내용을 바탕으로 사용자의 이용 서비스, 아이템 추천
    - ex) A는 주로 아이돌 장르를 듣고 그 중 트와이스 그룹의 노래를 듣는다.
    - 아이돌 장르 내 트와이스와 비슷한 음악을 하는 그룹들의 노래가 추천됨
- 단점
    - 사용자가 관심이 있는 내용만 추천하게 되기 때문에 항상 비슷한 내용만 추천할 수 있음
    - 오직 상용자가 이용하는 콘텐츠의 내용만 이용하여 추천이 되므로 추천의 품질 문제 발생

## 협업 필터링 방식
- 사용자의 서비스 이용 이력, 구매 이력과 같은 사용자 행동 양식만을 기반으로 추천을 수행
- 사용자-아이템 평점 행렬을 이용
- 사용자-아이템 평점 행렬에서 채워지지 않는 값에 대한 예측을 하여 사용자에게 적절한 아이템을 추천해줌


#### 사용자-아이템 평점 행렬
- 행이 사용자의 id로 구성
- 열이 아이템의 id로 구성
- 값은 사용자가 해당 item에 대해 매긴 평점
- 대부분의 값이 NaN으로 존재하는 희소 행렬의 특성

## 1) 최근접 이웃 협업 필터링(Nearst Neighbor)
- "사용자 기반 최근접 이웃 방식", "아이템 기반 최근접 이웃 방식" 존재
- "아이템 기반"이 보통 더 좋은 성능을 보임

### 1-1) 사용자 기반 최근접 이웃 방식
- 사용자별 데이터가 판단 기준
- 특정 사용자와 유사한 다른 사용자를 top-N만큼 추출하여 top-N사용자들이 좋아하는 아이템을 추천
- ex) 사용자 A벡터와 가장 유사한 벡터를 가지는 사용자 B벡터를 참고하여 영화를 추천

## 1-2) 아이템 기반 최근접 이웃 방식
- 아이템별 데이터가 판단의 기준
- 특정 아이템과 유사한 이용패턴을 보이는 다른 아이템들을 top-N방식으로 추천
- ex) A영화 열벡터와 가장 비슷한 패턴을 보이는 B벡터를 참조하여 A영화를 본 사용자에게 B영화를 추천

## 사용자-아이템 행렬의 NaN값 예측
1. SVD/NMF/SGD를 이용하여 행렬분해
2. 행렬분해를 통해 얻은 행렬을 통해 NaN값 예측
    - ex) 원 데이터의 (2,3)은 행렬분해를 통해 얻은 행렬들의 곱(2행과 3열의 내적)을 실시하여 생성

### SGD를 이용한 행렬분해

- 최적화 목적식  
$min\sum(r_{u,i}-p_{u}q_{i}^{t})^{2})+\lambda(||q_{i}||^{2}+||p_{u}||^{2})$

- $r_{u,i}$ : u행, i열의 원본 행렬값
- $p_{u}$ : p행렬의 u행
- $q_{i}$ : q행렬의 i행
- $\lambda$ : L2규제 계수(하이퍼 파라미터)

### SGD이용 행렬분해 예제

In [2]:
import numpy as np

R = np.array([[4,np.NaN,np.NaN,2,np.NaN],
             [np.NaN,5,np.NaN,3,1],
             [np.NaN,np.NaN,3,4,4],
             [5,2,1,2,np.NaN]])

num_users, num_items = R.shape
K = 3 # 잠재요인 수

In [4]:
# P, Q를 초기화
np.random.seed(1)
P = np.random.normal(scale = 1./K, size = (num_users,K))
Q = np.random.normal(scale = 1./K, size = (num_items,K))

In [6]:
from sklearn.metrics import mean_squared_error

def get_rmse(R,P,Q, non_zeros):
    error = 0
    
    # 예측행렬
    full_pred_matrix = np.dot(P,Q.T)
    
    # R에서 NaN이 아닌 값의 위치 인덱스 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    
    # NaN값이 아닌 값들을 추출
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind,y_non_zero_ind]
    
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [8]:
non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j]>0]

steps = 1000
learning_rate = 0.01
r_lambda = 0.01

for step in range(steps):
    for i,j,r in non_zeros:
        eij = r-np.dot(P[i,:],Q[j,:].T)
        
        # 업데이트
        P[i,:] += learning_rate * (eij*Q[j,:]-r_lambda*P[i,:])
        Q[j,:] += learning_rate * (eij*P[i,:]-r_lambda*Q[i,:])
    rmse = get_rmse(R,P,Q,non_zeros)
    if(step%50==0):
        print(step, rmse)

0 3.2217005473445837
50 0.48184344285378766
100 0.16070333559108133
150 0.07809887797145126
200 0.045118555491145004
250 0.02964811856740652
300 0.021934395535606412
350 0.018112408274720136
400 0.016242990802200083
450 0.015311962681727919
500 0.0148228277678383
550 0.014545027317045283
600 0.014372286173596415
650 0.014254352191819187
700 0.014166486815767987
750 0.014095979680917602
800 0.014036037962233124
850 0.013982902028095186
900 0.013934422769999798
950 0.013889331392406345


In [9]:
pred_matrix = np.dot(P,Q.T)
print(pred_matrix)

[[3.98927849 0.65581081 1.34258961 2.00657752 1.7123697 ]
 [6.89693211 4.98065727 0.99715213 2.98038613 0.99582561]
 [6.73376182 0.01273643 2.98353674 3.98012018 3.9932485 ]
 [4.97606056 2.00241607 1.00674715 2.00550696 1.11541337]]


In [10]:
print(R)

[[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]


### 캐글 영화데이터 세트를 통한 실습
- https://www.kaggle.com/tmdb-movie-metadata

In [11]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings("ignore")

In [12]:
movies = pd.read_csv("./tmdb_5000_movies.csv")
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [15]:
movies_df = movies[["id","title","genres","vote_average","vote_count",
                   "popularity","keywords","overview"]]
movies_df.head(1)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."


In [16]:
pd.set_option("max_colwidth",100)

In [17]:
from ast import literal_eval

movies_df["genres"] = movies_df["genres"].apply(literal_eval)

In [19]:
movies_df["keywords"] = movies_df["keywords"].apply(literal_eval)

In [20]:
movies_df["genres"] = movies_df["genres"].apply(lambda x:[y["name"]for y in x])

In [21]:
movies_df["keywords"] = movies_df["keywords"].apply(lambda x:[y["name"]for y in x])

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

movies_df["genres_literal"] = movies_df["genres"].apply(lambda x:(" ").join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df["genres_literal"])

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


In [27]:
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


In [25]:
def find_sim_movie(df,sorted_ind, title_name, top_n=10):
    title_movie = df[df["title"]==title_name]
    
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [28]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, "The Godfather",10)
similar_movies[["title","vote_average"]]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [29]:
movies_df[["title","vote_average","vote_count"]].sort_values("vote_average", ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


In [30]:
C = movies_df["vote_average"].mean()
m = movies_df["vote_count"].quantile(0.6)
print("C",round(C,3),"m",round(m,3))

C 6.092 m 370.2


In [31]:
def weighted_vote_average(record):
    v = record["vote_count"]
    R = record["vote_average"]
    
    return ((v/(v+m))*R + ((m/(m+v)))*C)

In [32]:
movies_df["weighted_vote"]=movies_df.apply(weighted_vote_average, axis=1)

In [33]:
movies_df[["title","vote_average","weighted_vote","vote_count"]].sort_values("weighted_vote",
                                                                            ascending=False)

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
...,...,...,...,...
2122,Epic Movie,3.2,4.737894,326
242,Fantastic Four,4.4,4.636554,2278
3746,The Boy Next Door,4.1,4.629738,1022
210,Batman & Robin,4.2,4.591725,1418


In [37]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    title_movie = df[df["title"]==title_name]
    title_index = title_movie.index.values
    
    similar_indexes = sorted_ind[title_index,:(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    return df.iloc[similar_indexes].sort_values("weighted_vote",ascending=False)[:top_n]

In [38]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, "The Godfather",10)
similar_movies[["title","vote_average", "weighted_vote"]]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
